In [1]:
import os
import json
import cv2
from tqdm import tqdm
from collections import defaultdict

gt_dir = '../../input/data/ICDAR19/ufo/train_gt_t13'
img_dir = '../../input/data/ICDAR19/images'

In [2]:
img_names = os.listdir(img_dir)
img_names = [img_name for img_name in img_names if not img_name.startswith('.')]
img_names = [img_name for img_name in img_names if img_name.split('.')[-1]!='gif'] # gif 제외
img_names.sort(key=lambda x:int(x.split('.')[0].split('_')[-1]))
img_names

['tr_img_00001.jpg',
 'tr_img_00002.jpg',
 'tr_img_00003.jpg',
 'tr_img_00004.jpg',
 'tr_img_00005.jpg',
 'tr_img_00006.jpg',
 'tr_img_00007.jpg',
 'tr_img_00008.jpg',
 'tr_img_00009.jpg',
 'tr_img_00010.jpg',
 'tr_img_00011.jpg',
 'tr_img_00012.jpg',
 'tr_img_00013.jpg',
 'tr_img_00014.jpg',
 'tr_img_00015.jpg',
 'tr_img_00016.jpg',
 'tr_img_00017.jpg',
 'tr_img_00018.jpg',
 'tr_img_00019.jpg',
 'tr_img_00020.jpg',
 'tr_img_00021.jpg',
 'tr_img_00022.jpg',
 'tr_img_00023.jpg',
 'tr_img_00024.jpg',
 'tr_img_00025.jpg',
 'tr_img_00026.jpg',
 'tr_img_00027.jpg',
 'tr_img_00028.jpg',
 'tr_img_00029.jpg',
 'tr_img_00030.jpg',
 'tr_img_00031.jpg',
 'tr_img_00032.jpg',
 'tr_img_00033.jpg',
 'tr_img_00034.jpg',
 'tr_img_00035.jpg',
 'tr_img_00036.jpg',
 'tr_img_00037.jpg',
 'tr_img_00038.jpg',
 'tr_img_00039.jpg',
 'tr_img_00040.jpg',
 'tr_img_00041.jpg',
 'tr_img_00042.jpg',
 'tr_img_00043.jpg',
 'tr_img_00044.jpg',
 'tr_img_00045.jpg',
 'tr_img_00046.jpg',
 'tr_img_00047.jpg',
 'tr_img_0004

In [7]:
# 오류: pd로 csv읽으니까 특수문자 ,를 나누라는 의미로 해석해서 안됨..
import pandas as pd
idx=str(1)
df = pd.read_csv(f'{gt_dir}/tr_img_{idx.zfill(5)}.txt',sep=',', header=None)
df

,0,1,2,3,4,5,6,7,8,9
0,33,1097,106,1083,91,1144,19,1142,Latin,CT
1,743,894,1082,850,1078,896,737,932,Arabic,تليفون:٠١٣٨٩٠٤٠٦٦
2,252,579,470,545,466,586,257,620,Latin,Systems
3,495,537,705,506,701,539,487,576,Latin,Division
4,408,419,773,380,763,449,393,515,Latin,Okaz
5,41,612,242,581,235,624,36,651,Latin,Security
6,98,430,258,342,221,484,66,538,Latin,O
7,823,343,1577,220,1581,303,829,436,Latin,Telecom
8,164,703,1435,507,1350,710,58,876,Latin,HIKVISION
9,255,362,392,338,379,481,229,495,Latin,###


In [9]:
# 저장할 json 객체 생성
new_json = {}
new_json['images'] = {}
new_json['version'] = {}
new_json['tags'] = {}

img_dir = img_dir

lang = defaultdict(set)

for img_name in tqdm(img_names):
    img = cv2.imread(os.path.join(img_dir,img_name))
    img_h = int(img.shape[0])
    img_w = int(img.shape[1])
    gt_name = img_name.split('.')[0] + '.txt' # 이미지에서 gt 파일명 생성

    new_json['images'][img_name] = {}

    new_json['images'][img_name]['paragraphs'] = None
    new_json['images'][img_name]['words'] = {}
    new_json['images'][img_name]['chars'] = None
    new_json['images'][img_name]['img_w'] = img_w
    new_json['images'][img_name]['img_h'] = img_h
    new_json['images'][img_name]['tags'] = None
    new_json['images'][img_name]['relations'] = None
    new_json['images'][img_name]['annotation_los'] = None
    new_json['images'][img_name]['license_tag'] = None

    f = open(os.path.join(gt_dir,gt_name),'r')
    lines = [list(line.strip().split(',')) for line in f.readlines()]
    f.close()
    for i,line in enumerate(lines):
        if len(line)>10:
            lines[i] = line[:9]
            lines[i].append(','.join(line[9:]))
        elif len(line)<10:
            print(line)

    idx = 1
    for line in lines:
        x1,y1,x2,y2,x3,y3,x4,y4,language,transcription = line
        bbox_idx = '0'*(4-len(str(idx))) + str(idx)
        new_json['images'][img_name]['words'][bbox_idx] = {}
        new_json['images'][img_name]['words'][bbox_idx]['transcription'] = transcription
        new_json['images'][img_name]['words'][bbox_idx]['points'] = [[x1,y1],[x2,y2],[x3,y3],[x4,y4]]
        new_json['images'][img_name]['words'][bbox_idx]['orientation'] = 'Horizontal'
        new_json['images'][img_name]['words'][bbox_idx]['language'] = [language] if language=='ko' or language=='en' else ['others']
        #if language=='Korean' or language=='korean' or language=='ko':
        lang[language].add(img_name)
        new_json['images'][img_name]['words'][bbox_idx]['tags'] = []
        new_json['images'][img_name]['words'][bbox_idx]['confidence'] = None
        new_json['images'][img_name]['words'][bbox_idx]['illegibility'] = False
        idx += 1

for k,v in lang.items():
    print(k,v)
    

100%|██████████| 9996/9996 [09:37<00:00, 17.31it/s] 

Latin {'tr_img_04148.jpg', 'tr_img_02081.jpg', 'tr_img_07403.jpg', 'tr_img_08871.jpg', 'tr_img_01744.jpg', 'tr_img_04236.jpg', 'tr_img_05229.jpg', 'tr_img_08445.jpg', 'tr_img_02736.jpg', 'tr_img_05927.jpg', 'tr_img_06069.jpg', 'tr_img_00402.jpg', 'tr_img_04954.jpg', 'tr_img_05919.jpg', 'tr_img_04948.jpg', 'tr_img_02435.jpg', 'tr_img_05449.jpg', 'tr_img_07091.jpg', 'tr_img_09093.jpg', 'tr_img_01643.jpg', 'tr_img_02429.jpg', 'tr_img_01071.jpg', 'tr_img_04564.jpg', 'tr_img_07241.jpg', 'tr_img_02245.jpg', 'tr_img_03161.jpg', 'tr_img_03885.jpg', 'tr_img_00970.jpg', 'tr_img_04830.jpg', 'tr_img_03436.jpg', 'tr_img_04485.jpg', 'tr_img_03166.jpg', 'tr_img_01488.jpg', 'tr_img_00111.png', 'tr_img_03211.jpg', 'tr_img_00433.jpg', 'tr_img_07442.jpg', 'tr_img_00769.jpg', 'tr_img_01074.jpg', 'tr_img_01168.jpg', 'tr_img_05930.jpg', 'tr_img_03856.jpg', 'tr_img_06637.jpg', 'tr_img_04170.jpg', 'tr_img_04517.jpg', 'tr_img_04956.jpg', 'tr_img_04993.jpg', 'tr_img_05799.jpg', 'tr_img_07376.jpg', 'tr_img_04397

In [11]:
with open('../../input/data/ICDAR19/ufo/ICDAR19.json','w') as f:
        json.dump(new_json, f, indent=4)